# 뉴스 NLP를 통한 알파 리서치

네이버 뉴스에서 종목명을 검색해 나오는 뉴스 기사들을 크롤링해 json으로 데이터셋화 시켜놨음. 

FnGuide의 KF-DeBERTa를 활용해 sentiment score 모델을 만들고 이 score을 통한 D-1 알파(하루 전 시그널로 다음 날 트레이딩) 전략을 만들어 백테스팅. 



## 1. 뉴스 데이터셋 불러오기

## 2. 뉴스 데이터셋 전처리

- 기사는 발행시간이 적혀있어 Point-in-Time 하므로 신뢰할 수 있는 데이터셋을 구축하는데 용이함. 
- 00시 자정을 기준으로 날짜를 나누는 것이 아닌, 장마감 시간을 바탕으로 date split
    - 장마감~익일 장시작전:
        - 익일 이용 가능한 데이터
    - 장시작~장마감
        - 당일 이용가능하나 look-ahead bias 있을 수 있음. 
- 본 분석에서는 아래에 나올 긍/부정 레이블링을 제대로 하기 위해 장마감~익일 장시작전 뉴스만 사용합니다. 

## 3. 뉴스 데이터셋 전처리 (심화)

네이버 뉴스에서 단순 종목명 기준으로 검색했기 때문에 대상 기업과 관련 없는 기업들의 기사가 섞여있을 가능성 농후함. 

예시:
- 셀트리온(068270): 이름이 고유하여 관련 없는 기사가 섞일 가능성 적음. 
- 대상(001680): 이름이 다른 문맥에서 발견되기 쉬움. 연예계 대상, 개인사업자 대상 대출, 등등

따라서 이 경우 NER(Named Entity Recognition) Task를 통해 기업/단체명에 해당되는 단어가 검출되는 기사만 남길 필요가 있음. 

## 4. 데이터셋 커버리지 확인

뉴스 데이터셋은 주가 데이터와 달리 모든 종목을 커버하지 못하며, 커버리지 내의 종목들 중에서도 잘 알려진 우량종목에 기사가 편중되어있을 가능성이 높음. 

즉, 알려진 주식에 대한 뉴스는 넘치나 잘 안알려진 주식에 대한 뉴스는 없거나 적을 것을 예상할 수 있음. 

아래와 같이 데이터 커버리지를 확인하여 이 사실을 확인할 수 있음. 

## 5. 뉴스 긍/부정 레이블링 

어떤 뉴스가 긍정적이냐 부정적이냐는 판단하기 어려운 문제. 한진칼(180640)의 경우 조양호 회장의 사망 이후 오히려 주가가 올랐고 ("긍정적" 뉴스), 엔씨소프트(036570)의 경우 TL(쓰론즈 앤 리버티) 신작을 발표한 이후 오히려 주가가 하락했음. ("부정적" 뉴스)

따라서 본 분석에선 그 날 주식시장의 등락이 장마감 후 뉴스 기사의 sentiment를 결정한다고 "가정"한다. 즉, 주식이 상승마감했을 경우 긍정 / 하락마감했을 경우 부정으로 기사를 labeling한다. 

단, 약보합 등의 미미한 변화에 대해 sentiment가 레이블링되어 모델에 노이즈가 더해지는 것을 막기 위해 일정 강도 이상의 상승/하락만 사용한다. (threshold 적용)

이 때, absolute threshold로 5%, 10% 이런 식으로 정하면 종목의 변동성 특성을 고려할 수 없기 때문에 20일 z-score을 이용하여 상승 하락을 time-series 방향에 대해 normalize한다. 
- 삼성전자(005930)와 같은 우량주는 코스닥 동전주보다 훨씬 변동성이 작을 것이므로, 5%만 움직여도 큰 뉴스일 수 있다. 
- time-series 방향으로 normalize하는 반면 cross-sectional 방향으로는 별도 normalization을 적용하지 않는다. 뉴스 기사의 긍부정을 레이블링하는데 주식의 고유 수익률이 필요하진 않기 때문. 


## 6. 뉴스 sentiment 모델 training

## 7. 뉴스 sentiment 모델 inference 

학습된 모델을 통해 뉴스데이터들의 sentiment score을 inference한다. 

## 8. 뉴스 sentiment 데이터셋 구축

앞에서 본 panel data 형식으로 news sentiment dataset을 구축한다. 